In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
import seaborn as sns
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [ ]:
df = pd.read_csv('market_basket_dataset.csv')
df.head()



In [ ]:
# Traduzindo e renomeando as colunas
df = df.rename(columns={
    'BillNo': 'Número_Fatura',
    'Itemname': 'Nome_Item',
    'Quantity': 'Quantidade',
    'Price': 'Preço',
    'CustomerID': 'ID_Cliente'
})


df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['Nome_Item'].unique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Calculando o total de vendas de cada item
df['Total_Vendas'] = df['Quantidade'] * df['Preço']

# Agrupando pelo nome do item e calculando o total de vendas de cada item
total_vendas_por_item = df.groupby('Nome_Item')['Total_Vendas'].sum()

total_vendas_por_item

In [ ]:
# Ordenando os resultados para encontrar os produtos menos vendidos
produtos_menos_vendidos = total_vendas_por_item.sort_values()

print("Produtos menos vendidos:")
print(produtos_menos_vendidos)

In [ ]:
# Calculando o valor total das compras de cada transação
df['Total'] = df['Quantidade'] * df['Preço']

# Calculando o valor total das compras de cada cliente
valor_total_por_cliente = df.groupby('ID_Cliente')['Total'].sum()

valor_total_por_cliente

In [ ]:
df.set_index('Número_Fatura', inplace=True)  # Definindo 'Número_Fatura' como o índice

# Calculando o valor total gasto por cada cliente (agora índice)
df['Total'] = df['Quantidade'] * df['Preço']
valor_total_gasto_por_cliente = df.groupby(df.index)['Total'].sum()

# Definindo segmentos com base no valor total gasto
def definir_segmento(valor):
    if valor > 100:
        return 'Alto'
    elif valor > 50:
        return 'Médio'
    else:
        return 'Baixo'

# Aplicando a função para criar uma nova coluna de segmento
df['Segmento'] = valor_total_gasto_por_cliente.apply(definir_segmento)

# Analisando o comportamento de compra de diferentes segmentos de clientes
comportamento_por_segmento = df.groupby('Segmento').agg({
    'Quantidade': 'sum',
    'Preço': 'mean',
    'Total': 'sum'
})

print(comportamento_por_segmento)


In [ ]:
# Contando o número de transações de cada cliente
frequencia_clientes = df.index.value_counts()

# Ordenando os resultados para encontrar os clientes mais frequentes
clientes_mais_frequentes = frequencia_clientes.sort_values(ascending=False)

print("Clientes mais frequentes:")
print(clientes_mais_frequentes)


In [ ]:
# Calculando o valor total gasto por cada cliente
valor_total_gasto_por_cliente = df.groupby(df.index)['Total'].sum()

# Ordenando os resultados para encontrar os clientes que gastam mais
clientes_que_gastam_mais = valor_total_gasto_por_cliente.sort_values(ascending=False)

print("Clientes que gastam mais:")
print(clientes_que_gastam_mais)


In [ ]:
# Calculando o preço médio, mínimo e máximo dos itens vendidos
preco_medio = df['Preço'].mean()
preco_minimo = df['Preço'].min()
preco_maximo = df['Preço'].max()

print("Preço médio dos itens vendidos:", preco_medio)
print("Preço mínimo dos itens vendidos:", preco_minimo)
print("Preço máximo dos itens vendidos:", preco_maximo)


In [ ]:
# Calculando o valor total de vendas em cada fatura
df['Valor_Total_Fatura'] = df['Quantidade'] * df['Preço']

# Agrupando por fatura e calculando o valor total de vendas em cada fatura
valor_total_por_fatura = df.groupby('Número_Fatura')['Valor_Total_Fatura'].sum()

print(valor_total_por_fatura)


In [ ]:
import matplotlib.pyplot as plt

# Calculando o número de itens por fatura
itens_por_fatura = df.groupby('Número_Fatura')['Quantidade'].sum()

# Visualizando a distribuição do número de itens por fatura
plt.figure(figsize=(10, 6))
plt.hist(itens_por_fatura, bins=20, color='skyblue', edgecolor='black')
plt.title('Distribuição do Número de Itens por Fatura')
plt.xlabel('Número de Itens')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()


In [ ]:
# Agrupando as transações por cliente e produto para contar quantas vezes cada produto foi comprado por cada cliente
preferencias_clientes = df.groupby(['ID_Cliente', 'Nome_Item']).size().reset_index(name='Quantidade_Comprada')

# Encontrando o produto mais comprado por cada cliente
preferencias_clientes = preferencias_clientes.sort_values(by=['ID_Cliente', 'Quantidade_Comprada'], ascending=[True, False])
preferencias_clientes = preferencias_clientes.groupby('ID_Cliente').first()

print(preferencias_clientes)


In [ ]:
# Agrupar os dados por nome do item e somar as quantidades vendidas
itens_populares = df.groupby('Nome_Item')['Quantidade'].sum()

# Ordenar os itens por quantidade vendida em ordem decrescente
itens_populares = itens_populares.sort_values(ascending=False)

# Selecionar os 10 itens mais populares
top_10_itens_populares = itens_populares.head(10)

print("Os 10 itens mais populares vendidos pela loja são:")
print(top_10_itens_populares)


In [ ]:
# Agrupa os dados por 'ID_Cliente' e calcula a média da 'Quantidade' e a soma do 'Preço' para cada cliente
comportamento_cliente = df.groupby('ID_Cliente').agg({'Quantidade': 'mean', 'Preço': 'sum'}).reset_index()

# Cria um DataFrame para exibir os valores
dados_tabela = pd.DataFrame({
    'ID_Cliente': comportamento_cliente['ID_Cliente'],
    'Quantidade Média': comportamento_cliente['Quantidade'],
    'Gasto Total': comportamento_cliente['Preço']
})

# Cria um subplot com um gráfico de dispersão e uma tabela
fig = go.Figure()

# Adiciona um gráfico de dispersão
fig.add_trace(go.Scatter(x=comportamento_cliente['Quantidade'], y=comportamento_cliente['Preço'],
                         mode='markers', text=comportamento_cliente['ID_Cliente'],
                         marker=dict(size=10, color='coral')))

# Adiciona uma tabela
fig.add_trace(go.Table(
    header=dict(values=['ID_Cliente', 'Quantidade Média', 'Gasto Total']),
    cells=dict(values=[dados_tabela['ID_Cliente'], dados_tabela['Quantidade Média'], dados_tabela['Gasto Total']]),
))

# Atualiza o layout
fig.update_layout(title='Comportamento do Cliente',
                  xaxis_title='Quantidade Média', yaxis_title='Gasto Total')

# Mostra o gráfico
fig.show()


In [ ]:
! pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Transforme os dados para o formato de cesta (basket format), onde cada linha representa uma transação
basket_data = df.groupby(['Número_Fatura', 'Nome_Item'])['Quantidade'].sum().unstack().reset_index().fillna(0).set_index('Número_Fatura')

# Converte a contagem em 0 ou 1 para cada item
basket_encoded = basket_data.applymap(lambda x: 1 if x > 0 else 0)

# Aplica o algoritmo Apriori para encontrar conjuntos frequentes de itens
frequent_itemsets = apriori(basket_encoded, min_support=0.05, use_colnames=True)

# Gera as regras de associação a partir dos conjuntos frequentes encontrados
association_rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Exibe as regras de associação encontradas
association_rules


In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Transforma os dados no formato de cesta, onde cada linha representa uma transação
dados_cesta = df.groupby(['Número_Fatura', 'Nome_Item'])['Quantidade'].sum().unstack().reset_index().fillna(0).set_index('Número_Fatura')

# Converte a contagem em 0 ou 1 para cada item
cesta_codificada = dados_cesta.applymap(lambda x: 1 if x > 0 else 0)

# Aplica o algoritmo Apriori para encontrar conjuntos frequentes de itens
conjuntos_frequentes = apriori(cesta_codificada, min_support=0.05, use_colnames=True)

# Gera regras de associação a partir dos conjuntos frequentes encontrados
regras_associacao = association_rules(conjuntos_frequentes, metric="lift", min_threshold=1)

# Exibe as regras de associação encontradas
print(regras_associacao)


In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Group items by BillNo and create a list of items for each bill
basket = df.groupby('Número_Fatura')['Nome_Item'].apply(list).reset_index()

# Encode items as binary variables using one-hot encoding
basket_encoded = basket['Nome_Item'].str.join('|').str.get_dummies('|')

# Find frequent itemsets using Apriori algorithm with lower support
frequent_itemsets = apriori(basket_encoded, min_support=0.01, use_colnames=True)

# Generate association rules with lower lift threshold
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=0.5)

# Display association rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))


In [ ]:
'''Vamos interpretar a saída passo a passo:

Antecedentes : São os itens considerados como ponto de partida ou “se” parte da regra de associação. Por exemplo, Pão, Manteiga, Cereais, Queijo e Frango são os antecedentes nesta análise.
Consequentes : São os itens que tendem a ser adquiridos junto com os antecedentes ou a parte “então” da regra de associação.
Suporte : O suporte mede a frequência com que uma determinada combinação de itens (antecedentes e consequentes) aparece no conjunto de dados. É essencialmente a proporção de transações em que os itens são comprados em conjunto. Por exemplo, a primeira regra indica que Pão e Maçãs são comprados juntos em aproximadamente 4,58% de todas as transações.
Confiança : A confiança quantifica a probabilidade do item consequente ser comprado quando o item antecedente já está na cesta. Em outras palavras, mostra a probabilidade de comprar o item consequente quando o item antecedente for comprado. Por exemplo, a primeira regra diz-nos que há 30,43% de probabilidade de comprar Maçãs quando o Pão já estiver no cesto.
Aumento : O aumento mede o grau de associação entre os itens antecedentes e consequentes, ao mesmo tempo que considera a probabilidade de compra básica do item consequente. Um valor de aumento superior a 1 indica uma associação positiva, o que significa que é mais provável que os itens sejam comprados juntos do que de forma independente. Um valor menor que 1 indica uma associação negativa. Por exemplo, a primeira regra tem um aumento de aproximadamente 1,86, sugerindo uma associação positiva entre Pão e Maçãs.'''

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Criar matriz de clientes e produtos
clientes_produtos = df.pivot_table(index='ID_Cliente', columns='Nome_Item', values='Quantidade', fill_value=0)

# Treinar modelo KNN
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(clientes_produtos)

# Encontrar os k vizinhos mais próximos para cada cliente
k = 5
distances, indices = knn_model.kneighbors(clientes_produtos, n_neighbors=k+1)

# Para cada cliente, recomendar produtos com base nos produtos comprados por seus vizinhos mais próximos
for i, cliente in enumerate(clientes_produtos.index):
    print(f"Cliente: {cliente}")
    print("Produtos recomendados:")
    
    # Conjunto para armazenar os produtos recomendados sem repetições
    produtos_recomendados_set = set()
    
    for j in indices[i][1:]:  # Excluindo o próprio cliente
        cliente_vizinho = clientes_produtos.index[j]
        produtos_recomendados_vizinho = df[df['ID_Cliente'] == cliente_vizinho]['Nome_Item'].unique()
        
        # Adicionar os produtos recomendados pelo vizinho ao conjunto
        for produto in produtos_recomendados_vizinho:
            produtos_recomendados_set.add(produto)
    
    # Exibir os produtos recomendados
    for produto in produtos_recomendados_set:
        print(f"- {produto}")
    
    print()


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split


# Configurar o dataframe para ser usado com a biblioteca Surprise
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[['ID_Cliente', 'Nome_Item', 'Quantidade']], reader)

# Dividir os dados em conjuntos de treinamento e teste
trainset, testset = train_test_split(data, test_size=0.2)

# Treinar o modelo de filtragem colaborativa (KNN)
model = KNNBasic(sim_options={'user_based': False})
model.fit(trainset)

# Para cada cliente, obter as recomendações de produtos
for cliente_id in df['ID_Cliente'].unique():
    produtos_recomendados = []
    for produto_id in df['Nome_Item'].unique():
        # Prever a classificação do produto para o cliente
        pred = model.predict(cliente_id, produto_id)
        # Se a classificação prevista for alta o suficiente, adicionar o produto às recomendações
        if pred.est > 0.5:
            produtos_recomendados.append(produto_id)
    # Exibir as recomendações para o cliente
    print(f"Cliente {cliente_id}: Recomendações - {produtos_recomendados}")
